In [1]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.models

from google.colab import drive

import numpy as np
import pandas as pd

import os
import time
import random
from collections import OrderedDict

In [2]:
drive.mount('/content/drive/')
!unzip -q drive/MyDrive/data/data5.zip -d data

Mounted at /content/drive/


In [3]:
%%bash

mkdir -p data/samples
cp drive/MyDrive/data/data7/labels.csv ./

touch log.txt

counter=0
for file in drive/MyDrive/data/data7/samples/*
do
    if (( counter >= 15000 ))
    then break
    fi

    cp "$file" data/samples/
    ((counter++))
done

mv data/labels.csv labels2.csv

In [4]:
labels1 = pd.read_csv('labels.csv')
labels2 = pd.read_csv('labels2.csv')
labels3 = pd.concat([labels1, labels2], axis=0)
labels4 = labels3.drop(['Unnamed: 0'], axis=1)
labels4.to_csv('data/labels.csv')

In [5]:

class SpatialAttention(nn.Module):

  def __init__(self):
    super(SpatialAttention, self).__init__()
  
  def forward(self, x_q, x_k, x_v):

    b, c, t, h, w = x_q.size()

    r_q = x_q.permute((0, 2, 1, 3, 4)).reshape(b*t, c, h*w)
    r_k = x_k.permute((0, 2, 1, 3, 4)).reshape(b*t, c, h*w)
    r_v = x_v.permute((0, 2, 1, 3, 4)).reshape(b*t, c, h*w)

    m_c = T.matmul(r_q, r_k.transpose(1, 2))
    m_c = F.softmax(m_c, dim=0)
    a_c = T.matmul(m_c, r_v)

    m_s = T.matmul(r_q.transpose(1, 2), r_k)
    m_s = F.softmax(m_s, dim=0)
    a_s = T.matmul(m_s, r_v.transpose(1, 2)).transpose(1, 2)

    a = a_c + a_s
    a = a.reshape(b, t, c, h, w).permute((0, 2, 1, 3, 4))
    return a


class TemporalAttention(nn.Module):

  def __init__(self):
    super(TemporalAttention, self).__init__()
  
  def forward(self, x, x_v):

    b, c, t, h, w = x.size()

    r_q = x.permute((0, 2, 1, 3, 4)).reshape(b, t, c*h*w)
    r_k = r_q.transpose(1, 2)
    r_v = x_v.permute((0, 2, 1, 3, 4)).reshape(b, t, c*h*w)

    m_t = T.matmul(r_q, r_k)
    m_t = F.softmax(m_t, dim=0)
    a_t = T.matmul(m_t, r_v)

    a = a_t.reshape(b, t, c, h, w).permute((0, 2, 1, 3, 4))
    return a


class SSA(nn.Module):

  def __init__(self, in_dim, out_dim):
    super(SSA, self).__init__()

    self.x_q_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(1, 1, 1))
    self.x_k_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(1, 1, 1))
    self.x_v_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(1, 1, 1))
    self.spatial = SpatialAttention()

    self.X_embed = nn.Conv3d(in_dim, out_dim, kernel_size=(3, 1, 1), padding=(1, 0, 0))
    self.temporal = TemporalAttention()
  
  def forward(self, x):
    '''
      x = T.tensor of shape (B, C, T, H, W)
    '''

    x_q = F.relu(self.x_q_embed(x))
    x_k = F.relu(self.x_k_embed(x))
    x_v = F.relu(self.x_v_embed(x))
    s = self.spatial(x_q, x_k, x_v)

    X = F.relu(self.X_embed(s))
    t = self.temporal(X, x_v)

    return t


In [6]:

class IdBlock(nn.Module):

  def __init__(self, in_dim, hidden_dim, kernel_size):
    super(IdBlock, self).__init__()

    self.conv1 = nn.Conv3d(in_dim, hidden_dim, 1)
    self.norm1 = nn.InstanceNorm3d(hidden_dim)

    self.conv2 = nn.Conv3d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding='same')
    self.norm2 = nn.InstanceNorm3d(hidden_dim)

    self.conv3 = nn.Conv3d(hidden_dim, in_dim, 1)
    self.norm3 = nn.InstanceNorm3d(in_dim)

  def forward(self, x):

    x_shortcut = x

    x = self.conv1(x)
    x = self.norm1(x)
    x = F.relu(x)

    x = self.conv2(x)
    x = self.norm2(x)
    x = F.relu(x)

    x = self.conv3(x)
    x = self.norm3(x)

    x = x + x_shortcut
    x = F.relu(x)
    
    return x


class ConvBlock(nn.Module):

  def __init__(self, in_dim, hidden_dim, out_dim, kernel_size, stride=1):
    super(ConvBlock, self).__init__()

    self.conv0 = nn.Conv3d(in_dim, out_dim, 1, stride=stride)
    self.norm0 = nn.InstanceNorm3d(out_dim)

    self.conv1 = nn.Conv3d(in_dim, hidden_dim, 1, stride=stride)
    self.norm1 = nn.InstanceNorm3d(hidden_dim)

    self.conv2 = nn.Conv3d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding='same')
    self.norm2 = nn.InstanceNorm3d(hidden_dim)

    self.conv3 = nn.Conv3d(hidden_dim, out_dim, 1)
    self.norm3 = nn.InstanceNorm3d(out_dim)
  
  def forward(self, x):

    x_shortcut = self.conv0(x)
    x_shortcut = self.norm0(x_shortcut)

    x = self.conv1(x)
    x = self.norm1(x)
    x = F.relu(x)

    x = self.conv2(x)
    x = self.norm2(x)
    x = F.relu(x)

    x = self.conv3(x)
    x = self.norm3(x)

    x = x + x_shortcut
    x = F.relu(x)
    
    return x


class SSABlock(nn.Module):

  def __init__(self, in_dim, hidden_dim, kernel_size):
    super(SSABlock, self).__init__()

    self.conv1 = nn.Conv3d(in_dim, hidden_dim, 1)
    self.norm1 = nn.InstanceNorm3d(hidden_dim)

    self.ssa2 = SSA(hidden_dim, hidden_dim)

    self.conv3 = nn.Conv3d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding='same')
    self.norm3 = nn.InstanceNorm3d(hidden_dim)

    self.conv4 = nn.Conv3d(hidden_dim, in_dim, 1)
    self.norm4 = nn.InstanceNorm3d(in_dim)

  def forward(self, x):

    x_shortcut = x

    x = self.conv1(x)
    x = self.norm1(x)
    x = F.relu(x)

    x = self.ssa2(x)

    x = self.conv3(x)
    x = self.norm3(x)
    x = F.relu(x)

    x = self.conv4(x)
    x = self.norm4(x)

    x = x + x_shortcut
    x = F.relu(x)
    
    return x
  

In [7]:

class Model(nn.Module):

  def __init__(self):
    super(Model, self).__init__()

    self.conv1 = nn.Sequential(OrderedDict([
        ('conv1a', nn.Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3))),
        ('conv1b', nn.InstanceNorm3d(64)),
        ('conv1c', nn.ReLU()),
        ('conv1d', nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))),
    ]))

    self.res2 = nn.Sequential(OrderedDict([
        ('res2a', ConvBlock(64, 64, 256, kernel_size=(1, 3, 3))),
        ('res2b', SSABlock(256, 64, kernel_size=(1, 3, 3))),
        ('res2c', SSABlock(256, 64, kernel_size=(1, 3, 3))),
    ]))

    self.res3 = nn.Sequential(OrderedDict([
        ('res3a', ConvBlock(256, 128, 512, kernel_size=(1, 3, 3), stride=(1, 2, 2))),
        ('res3b', SSABlock(512, 128, kernel_size=(1, 3, 3))),
        ('res3c', SSABlock(512, 128, kernel_size=(1, 3, 3))),
        ('res3d', SSABlock(512, 128, kernel_size=(1, 3, 3))),
    ]))

    self.res4 = nn.Sequential(OrderedDict([
        ('res4a', ConvBlock(512, 256, 1024, kernel_size=(3, 3, 3), stride=(1, 2, 2))),
        ('res4b', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4c', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4d', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4e', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
        ('res4f', IdBlock(1024, 256, kernel_size=(3, 3, 3))),
    ]))

    self.res5 = nn.Sequential(OrderedDict([
        ('res5a', ConvBlock(1024, 512, 2048, kernel_size=(3, 3, 3), stride=(2, 2, 2))),
        ('res5b', IdBlock(2048, 512, kernel_size=(3, 3, 3))),
        ('res5c', IdBlock(2048, 512, kernel_size=(3, 3, 3))),
    ]))

    self.fc6 = nn.Sequential(OrderedDict([
        ('fc6a', nn.AvgPool3d(kernel_size=(1, 7, 7))),
        ('fc6b', nn.Flatten(start_dim=1)),
        ('fc6c', nn.Dropout(p=0.3)),
        ('fc6d', nn.Linear(8192, 1024)),
        ('fc6e', nn.ReLU()),
        ('fc6f', nn.Dropout(p=0.3)),
        ('fc6g', nn.Linear(1024, 1)),
    ]))
    
  
  def forward(self, x):

    x = self.conv1(x)
    x = self.res2(x)
    x = self.res3(x)
    x = self.res4(x)
    x = self.res5(x)
    x = self.fc6(x)

    return x



In [8]:

class Data(Dataset):

  def __init__(self, samples, labels):
    self.samples = samples
    self.labels = labels
    self.len = samples.size(0)
  
  def __getitem__(self, index):
    return self.samples[index, :, :], self.labels[index, :]

  def __len__(self):
    return self.len


class LazyData(Dataset):

  def __init__(self, samples_path, labels_path):
    self.samples_path = samples_path
    self.labels_path = labels_path
    self.samples_list = [
      filename for filename in 
      os.listdir(samples_path) if 
      filename != '.DS_Store'
    ]
    labels_df = pd.read_csv(labels_path)
    self.labels_dict = {
      row['filename']: 
      T.tensor([row['label']], dtype=T.float)
      for _, row in labels_df.iterrows()
    }
    self.len = len(self.samples_list)
  
  def __getitem__(self, index):
    filename = self.samples_list[index]
    sample_path = os.path.join(self.samples_path, filename)
    sample = T.load(sample_path)
    label = self.labels_dict[filename]
    return sample, label

  def __len__(self):
    return self.len


class SmallData(Dataset):

  def __init__(self, samples_path, labels_path, n_samples=None):
    self.samples_path = samples_path
    self.labels_path = labels_path
    self.samples_list = [
      filename for filename in 
      os.listdir(samples_path) if 
      filename != '.DS_Store'
    ]
    if n_samples is not None:
      self.samples_list = random.choices(self.samples_list, k=n_samples)
    labels_df = pd.read_csv(labels_path)
    self.labels_dict = {
      row['filename']: 
      T.tensor([row['label']], dtype=T.float)
      for _, row in labels_df.iterrows()
      if row['filename'] in self.samples_list
    }
    self.len = len(self.samples_list)
  
  def __getitem__(self, index):
    filename = self.samples_list[index]
    sample_path = os.path.join(self.samples_path, filename)
    sample = T.load(sample_path)
    label = self.labels_dict[filename]
    return sample, label

  def __len__(self):
    return self.len


class FakeData(Dataset):

  def __init__(self, n_samples=1000):
    self.samples = [T.rand((3, 8, 224, 224)) for _ in range(n_samples)]
    self.labels = [T.rand((1,)) for _ in range(n_samples)]
    self.len = n_samples
  
  def __getitem__(self, index):
    sample = self.samples[index]
    label = self.labels[index]
    return sample, label

  def __len__(self):
    return self.len
  


In [9]:
class EarlyStopping():

  def __init__(self, max_epochs=1, patience=1, tolerance=0):

    self.max_epochs = max_epochs
    self.max_epochs_counter = 0
    self.patience = patience
    self.patience_counter = 0
    self.tolerance = tolerance
    self.best = float('inf')

  def __call__(self, val_loss):

    if val_loss < self.best + self.tolerance:
      self.patience_counter = 0
      if self.tolerance == 0 or val_loss < self.best:
        self.best = val_loss
    else:
      self.patience_counter += 1
    self.max_epochs_counter += 1
    return (self.patience_counter == self.patience 
            or self.max_epochs_counter == self.max_epochs)


class ModelSaving():

  def __init__(self, save_path):
    self.save_path = save_path
    self.best = float('inf')

  def __call__(self, model, val_loss):
    if val_loss < self.best:
      self.best = val_loss
      T.save(model.state_dict(), self.save_path)

In [10]:
class Trainer():

  def __init__(self, device, model, criterion, optimizer, dataset,
               prop_val=0, prop_test=0, batch_size=1, num_workers=0, 
               scheduler=None, saver=None, stopper=None):

    self.device = device
    self.model = model
    self.criterion = criterion
    self.optimizer = optimizer
    self.scheduler = scheduler
    self.saver = saver
    self.stopper = stopper or EarlyStopping()
    self.epoch = 0
    
    len_val = int(prop_val * len(dataset))
    len_test = int(prop_test * len(dataset))
    len_train = len(dataset) - len_val - len_test
    train, val, test = random_split(
      dataset, [len_train, len_val, len_test],
      generator=T.Generator().manual_seed(4)
    )
    self.train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    self.val_loader = DataLoader(val, batch_size=batch_size, num_workers=num_workers)
    self.test_loader = DataLoader(test, batch_size=batch_size, num_workers=num_workers)
  
  def train_step(self):
    self.optimizer.zero_grad()
    total_loss = 0
    len_counter = 0
    dataloader = self.train_loader
    for x, y in dataloader:
      x = x.to(self.device)
      y = y.to(self.device)
      pred = self.model(x)
      loss = self.criterion(pred, y)
      loss.backward()
      total_loss += x.size(0) * loss.item()
      len_counter += x.size(0)
      self.optimizer.step()
      self.optimizer.zero_grad()
    total_loss /= len_counter
    return total_loss

  def eval_step(self, test=False):
    total_loss = 0
    len_counter = 0
    dataloader = self.test_loader if test else self.val_loader
    with T.no_grad():
      for x, y in dataloader:
        x = x.to(self.device)
        y = y.to(self.device)
        pred = self.model(x)
        loss = self.criterion(pred, y)
        total_loss += x.size(0) * loss.item() 
        len_counter += x.size(0)
    total_loss /= len_counter
    return total_loss

  def train_loop(self, verbose=False):

    if verbose:
      print(self.model)
      print(f'Using device: {device0}.')
      since = time.perf_counter()

    while True:
      self.epoch += 1

      self.model = self.model.train()
      train_loss = self.train_step()

      self.model = self.model.eval()
      val_loss = self.eval_step()

      if verbose:
        if self.epoch == 1:
          free_mem, total_mem = T.cuda.mem_get_info(self.device)
          print(f'{free_mem/(1024**2):.1f} MB free of {total_mem/(1024**2):.1f} MB.')
        print(
          f'{time.perf_counter() - since:>6.1f}s :: Epoch {self.epoch} complete. ' +
          f'(train: {train_loss:.3f}, val: {val_loss:.3f}).'
        )
        since = time.perf_counter()

      if self.saver: self.saver(self.model, val_loss)
      if self.stopper(val_loss): break
      if self.scheduler: self.scheduler.step(val_loss)
    
    if self.saver:
      best_state = T.load(self.saver.save_path)
      self.model.load_state_dict(best_state)

    self.model = self.model.eval()
    test_loss = self.eval_step(test=True)

    if verbose: print(f'Test loss: {test_loss}.')



In [11]:

device0 = T.device('cuda:0' if T.cuda.is_available() else 'cpu')

dataset0 = LazyData(
    os.path.join('data', 'samples'),
    os.path.join('data', 'labels.csv'),
)

model0 = Model()
model0 = model0.eval().to(device0)

optimizer0 = T.optim.Adam(model0.parameters(), lr=0.001)
scheduler0 = T.optim.lr_scheduler.ReduceLROnPlateau(
  optimizer0, factor=0.3, verbose=True, patience=8
)
criterion0 = nn.MSELoss()

saver0 = ModelSaving('model')
stopper0 = EarlyStopping(max_epochs=60, patience=12)

In [ ]:
trainer0 = Trainer(
  device0, model0, criterion0, optimizer0, dataset0,
  prop_val=0.2, prop_test=0.2, batch_size=4, num_workers=3, 
  scheduler=scheduler0, saver=saver0, stopper=stopper0
)
trainer0.train_loop(verbose=True)

Model(
  (conv1): Sequential(
    (conv1a): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3))
    (conv1b): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv1c): ReLU()
    (conv1d): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
  )
  (res2): Sequential(
    (res2a): ConvBlock(
      (conv0): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1))
      (norm0): InstanceNorm3d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1))
      (norm1): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv2): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=same)
      (norm2): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1)

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive/')
! mv model drive/MyDrive/
drive.flush_and_unmount()